## Step 1: Read Data File
Import necessary packages.

In [1]:
import pandas as pd #for working with dataframe
import numpy as np
from scipy import stats, integrate #will help in stats operation in histograms
import matplotlib.pyplot as plt #to plot the PE
import seaborn as sns #nice looking plots
%matplotlib inline
sns.set(color_codes=True)

In [2]:
df=pd.read_csv("Flatdata_Final.csv",encoding='latin1')

## Step 2: Find a unique identifier for each project.
Some of the records in the database don't have any Internal Project ID number. We will need to analyse these differently. Create a new dataframe with only the rows that have a InternalProjectID.
### Create a new dataframe with InternalProjectIDs.
#### flatdata_withID contains the rows that have project IDs.

In [3]:
flatdata_withID=df[df['InternalProjectID'].notnull()]
flatdata_withID #dataframe with IDs available.

,BriefDescription,ProjectYear,InternalProjectID,ImprovementType,AreaTypeFuncClass,main_ProjectInformation.State,TollType,TrafficForecast,ForecastYear,ForecastYearType,...,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,main_TrafficCounts.State,CounterID,FunctionalClass
0,NaN,2006.0,2320,NaN,3.0,WI,No Tolls on 1+ lane,34000,2007.0,1.0,...,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,Wisconsin,58-22417-401399,3-Principal Arterial
1,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,30000,2011.0,1.0,...,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,Wisconsin,58-22419-404512,3-Principal Arterial
2,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,19000,2011.0,1.0,...,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,Wisconsin,58-22421-404513,3-Principal Arterial
3,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,21900,2011.0,1.0,...,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,Wisconsin,58-22423-404524,4-Minor Arterial
4,NaN,2006.0,2322,NaN,3.0,WI,No Tolls on 1+ lane,22700,2009.0,1.0,...,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,Wisconsin,58-22425-401843,3-Principal Arterial
5,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,2800,2011.0,1.0,...,NaN,58-6307-2467-,NaN,58-22427-404541,2011,2500,404541.0,Wisconsin,58-22427-404541,5-Major Collector
6,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,33500,2011.0,1.0,...,NaN,58-6308-2467-,NaN,58-22429-404545,2011,33500,404545.0,Wisconsin,58-22429-404545,3-Principal Arterial
7,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,1500,2011.0,1.0,...,NaN,58-6309-2467-,NaN,58-22431-404547,2011,2200,404547.0,Wisconsin,58-22431-404547,7-Local
8,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,23800,2011.0,1.0,...,NaN,58-6310-2467-,NaN,58-22433-404548,2011,25400,404548.0,Wisconsin,58-22433-404548,3-Principal Arterial
9,NaN,2006.0,2373,NaN,1.0,WI,No Tolls on 1+ lane,8600,2008.0,1.0,...,NaN,58-6311-2373-,NaN,58-22435-510108,2008,6600,510108.0,Wisconsin,58-22435-510108,3-Principal Arterial


#### flatdata_withoutID don't have any ID. will have to be cleaned up first differently.
In this flatdata_withoutID there are projects from Ohio and Florida D-4.
The Florida D-4 data just needs minor tinkering in the brief-description column. We will take the second '-' sign and then take the characters before that to be the unique project ID.


In [4]:
flatdata_withoutID=df[df['InternalProjectID'].isnull()]

Exclude Ohio (State ID= 41) dataset from flatdata_withID. They don't actually indicate whether the actual counts are taken after the project was completed or not, that is why the acccuracy assessment may not be entirely correct. Looking at the Ohio dataset, we see that in the original excel worksheetthere is a column titled "Physical Works Completed On". If the Near Opening Year Count happens before this date, that means the count is taken when the project is not yet done. SO we have to exclude these values. Ohio dataset will have to be analysed separately.
##### Ohio dataset cleaned up in a different way and then added to flatdata_withID

In [6]:
flatdata_withID=df[df['main_ProjectInformation.State']!='OH']

##### Excluding Florida as well. As we are not sure if the traffic counts are taken after the project has been completed or not. 
Need to apply the same logic as before i.e. buffer of 1 or 2 years.

In [7]:
flatdata_withID=flatdata_withID[flatdata_withID['main_ProjectInformation.State']!='FL']

## Step 3: Apply filters to get useful rows

#### 3.1 Filter out the rows where forecast year and year of observation is the same
This filter will give the rows whose forecasts we don't need to adjust.

In [9]:
flatdata_adjusted=flatdata_withID[flatdata_withID['Year_ofObservation']==flatdata_withID['ForecastYear']]

flatdata_adjusted

,BriefDescription,ProjectYear,InternalProjectID,ImprovementType,AreaTypeFuncClass,main_ProjectInformation.State,TollType,TrafficForecast,ForecastYear,ForecastYearType,...,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,main_TrafficCounts.State,CounterID,FunctionalClass
0,NaN,2006.0,2320,NaN,3.0,WI,No Tolls on 1+ lane,34000,2007.0,1.0,...,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,Wisconsin,58-22417-401399,3-Principal Arterial
1,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,30000,2011.0,1.0,...,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,Wisconsin,58-22419-404512,3-Principal Arterial
2,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,19000,2011.0,1.0,...,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,Wisconsin,58-22421-404513,3-Principal Arterial
3,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,21900,2011.0,1.0,...,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,Wisconsin,58-22423-404524,4-Minor Arterial
4,NaN,2006.0,2322,NaN,3.0,WI,No Tolls on 1+ lane,22700,2009.0,1.0,...,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,Wisconsin,58-22425-401843,3-Principal Arterial
5,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,2800,2011.0,1.0,...,NaN,58-6307-2467-,NaN,58-22427-404541,2011,2500,404541.0,Wisconsin,58-22427-404541,5-Major Collector
6,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,33500,2011.0,1.0,...,NaN,58-6308-2467-,NaN,58-22429-404545,2011,33500,404545.0,Wisconsin,58-22429-404545,3-Principal Arterial
7,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,1500,2011.0,1.0,...,NaN,58-6309-2467-,NaN,58-22431-404547,2011,2200,404547.0,Wisconsin,58-22431-404547,7-Local
8,NaN,2007.0,2467,NaN,3.0,WI,No Tolls on 1+ lane,23800,2011.0,1.0,...,NaN,58-6310-2467-,NaN,58-22433-404548,2011,25400,404548.0,Wisconsin,58-22433-404548,3-Principal Arterial
9,NaN,2006.0,2373,NaN,1.0,WI,No Tolls on 1+ lane,8600,2008.0,1.0,...,NaN,58-6311-2373-,NaN,58-22435-510108,2008,6600,510108.0,Wisconsin,58-22435-510108,3-Principal Arterial


#### The rows where the Year of Observation is later than the forecast year will need to be adjusted.

In [10]:
flatdata_unadjusted=flatdata_withID[flatdata_withID['Year_ofObservation']>flatdata_withID['ForecastYear']]

### Step 3.3: Filter out by Forecast Year Type
Since we are only looking at Opening Year forecasts (Year Type ID= 1), filter out the previous dataset by Type 1.

In [11]:
flatdata_adjusted=flatdata_adjusted[flatdata_adjusted['ForecastYearType']==1.0]

In [12]:
flatdata_adjusted=flatdata_adjusted.assign(AdjustedForecast=flatdata_withID['TrafficForecast'])

## Step 4: Clean up Ohio, Florida and European Datasets
These datasets are actually cleaned up from the database each agency provided, since there were many errors while exporting from the excel to the database as well as data-definition and deficiency. We appended the cleaned up datasets with the flatdata_adjusted dataset. 

For Florida and Ohio, the "Target Year", or the year we are comparing the actual traffic with the forecasted traffic is taken based a buffer created around the forecast year: 1 year buffer for low-risk projects (on existing roadways which are assumed to be completed within 1 year of forecast) and a 2 year buffer for high-risk ones (new construction projects, which are assumed to take at least 2 years more than forecast)

In [13]:
def TargetYear(x): #get the year to compare
    if x.ImprovementType==1: #low risk projects
        year=x.ForecastYear+1
    else:
        year=x.ForecastYear+2 #buffer of 2 years.
    
    return year

##  Step 5: Adjust Traffic Forecast to target Year
In the database we have forecast information for Opening Year and Design year. We interpolated the traffic forecast to the Target Year

In [14]:
def TrafficForecast(x): #to adjust the forecast. Linearly interpolated
    m=(x.DYADTForecast-x.OYADTForecast)/(x.DesignYear-x.OpeningYear)
    adjustedforecast=m*(x.Year-x.OpeningYear)+x.OYADTForecast
    return adjustedforecast

## Final, Cleaned Up Dataset
This dataset also contains unemployment rate information, which were added separately with information from Bureau of Labor Statistics for the Year data in each record for each state. 

In [15]:
flatdata_adjusted=pd.read_csv('flatdata_coded.csv',encoding='latin-1')
pd.options.display.max_columns=50

## Step 6: Create Columns for Analyses Variables
1. DiffYear= Difference between the Year actual count was available and the Year Forecast was Produced
2. Forecast Horizon= Dependent on DiffYear Variable, categorical variable.
3. Error= Coined as Actual Traffic Count minus the Forecasted Traffic Volume
4. Percent Error= Error over the Forecasted Traffic Volume.
5. AbsPE= Absolute Value of the Percent Error

In [16]:
flatdata_adjusted=flatdata_adjusted.assign(DiffYear=flatdata_adjusted['Year_ofObservation']-flatdata_adjusted['YearForecastProduced'])
flatdata_adjusted['ForecastHorizon'] = np.where(flatdata_adjusted['DiffYear']<=5, flatdata_adjusted['DiffYear'], '5+')
flatdata_adjusted=flatdata_adjusted.assign(Error=flatdata_adjusted['Count']-flatdata_adjusted['AdjustedForecast'])
flatdata_adjusted=flatdata_adjusted.assign(PercentError=100*flatdata_adjusted['Error']/flatdata_adjusted['AdjustedForecast'])
flatdata_adjusted=flatdata_adjusted.assign(AbsPE=np.abs(flatdata_adjusted['PercentError']))
flatdata_adjusted

,BriefDescription,InternalProjectID,ImprovementType,ImprovementType_Availability,AreaTypeFuncClass,AreaTypeFuncClass_Availability,State,Agency,TollType,TrafficForecast,ForecastYear,ForecastYear_Availability,ForecastYearType,YearForecastProduced,YearForecastProduced_Availability,ForecasterType,ForecastingType_Availability,ForecastMethodology,ForecastMethodology_Availability,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,CounterID,FunctionalClass,FunctionalClass_Availability,DiffYear_Availability,AdjustedForecast,UnemploymentRate_ForecastYear,UnemploymentRate_ForecastYear_Availability,UnemploymentRate_YearProduced,UnemploymentRate_YearProduced_Availability,UnemploymentRate_OpeningYear,UnemploymentRate_OpeningYear-Availability,DiffYear,ForecastHorizon,Error,PercentError,AbsPE
0,NaN,2320,12,0,3,1,WI,F,No Tolls on 1+ lane,34000,2007,1,1,2006,1,1,1,1,1,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,58-22417-401399,3,1,1,34000.0,4.90,1,4.70,1,4.90,1,1,1,-2500.0,-7.352941,7.352941
1,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,30000,2011,1,1,2007,1,1,1,1,1,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,58-22419-404512,3,1,1,30000.0,7.80,1,4.90,1,7.80,1,4,4,-7800.0,-26.000000,26.000000
2,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,19000,2011,1,1,2007,1,1,1,1,1,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,58-22421-404513,3,1,1,19000.0,7.80,1,4.90,1,7.80,1,4,4,1100.0,5.789474,5.789474
3,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,21900,2011,1,1,2007,1,1,1,1,1,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,58-22423-404524,4,1,1,21900.0,7.80,1,4.90,1,7.80,1,4,4,-3500.0,-15.981735,15.981735
4,NaN,2322,12,0,3,1,WI,F,No Tolls on 1+ lane,22700,2009,1,1,2006,1,1,1,1,1,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,58-22425-401843,3,1,1,22700.0,8.60,1,4.70,1,8.60,1,3,3,-100.0,-0.440529,0.440529
5,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,2800,2011,1,1,2007,1,1,1,1,1,NaN,58-6307-2467-,NaN,58-22427-404541,2011,2500,404541.0,58-22427-404541,5,1,1,2800.0,7.80,1,4.90,1,7.80,1,4,4,-300.0,-10.714286,10.714286
6,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,33500,2011,1,1,2007,1,1,1,1,1,NaN,58-6308-2467-,NaN,58-22429-404545,2011,33500,404545.0,58-22429-404545,3,1,1,33500.0,7.80,1,4.90,1,7.80,1,4,4,0.0,0.000000,0.000000
7,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,1500,2011,1,1,2007,1,1,1,1,1,NaN,58-6309-2467-,NaN,58-22431-404547,2011,2200,404547.0,58-22431-404547,7,1,1,1500.0,7.80,1,4.90,1,7.80,1,4,4,700.0,46.666667,46.666667
8,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,23800,2011,1,1,2007,1,1,1,1,1,NaN,58-6310-2467-,NaN,58-22433-404548,2011,25400,404548.0,58-22433-404548,3,1,1,23800.0,7.80,1,4.90,1,7.80,1,4,4,1600.0,6.722689,6.722689
9,NaN,2373,12,0,1,1,WI,F,No Tolls on 1+ lane,8600,2008,1,1,2006,1,1,1,1,1,NaN,58-6311-2373-,NaN,58-22435-510108,2008,6600,510108.0,58-22435-510108,3,1,1,8600.0,4.90,1,4.70,1,4.90,1,2,2,-2000.0,-23.255814,23.255814


### 6.1 Functions for Categorical Variables

In [17]:
def decade_produced(x): #Variable for calculating the decade forecast was produced
    if x.YearForecastProduced<=1980:
        decade=1
    elif x.YearForecastProduced<=1990:
        decade=2
    elif x.YearForecastProduced<=2000:
        decade=3
    else:
        decade=5
    return decade
def agency(x): #Variable to mask the agency
    if x.State=='FL':
        agency='A'
    elif x.State=='MA':
        agency='B'
    elif x.State=='MI':
        agency='C'
    elif x.State=='MN':
        agency='D'
    elif x.State=='OH':
        agency='E'
    elif x.State=='WI':
        agency='F'
    else:
        agency='European Project'
    return agency
def areatype(x): #Variable for Project Area Type
    if x.AreaTypeFuncClass<3: #Area Type 1 and 2 are Rural and Mostly Rural Area
        area=1
    elif x.AreaTypeFuncClass<4: #Area Type 3 is Urban
        area=2
    else: #Area Type 4 is Unknown
        area=3
    return area
def decade_opening(x): #Variable for aggregating over the Project Forecast Year
    if x.ForecastYear<=1990: #Projects that are forecasted to open before 1990
        decade2=1
    elif x.ForecastYear<=2000: #Projects that are forecasted to open between 1990 and 2000
        decade2=2
    elif x.ForecastYear<=2002: #To account for the Recession in 2001. 2001 and 2002 was taken as the years of influence
        decade2=3   
    elif x.ForecastYear<=2008: 
        decade2=4
    elif x.ForecastYear<=2012: #To account for the recession in 2008, the effect of which continued till 2012
        decade2=5
    else:
        decade2=6
    return decade2
def decade_count(x): #Variable for aggregating over the Year of Observation, same as before.
    if x.Year_ofObservation<=1990:
        decade3='1'
    #elif x.Year_ofObservation<=2000:
        #decade3='2'
    elif x.Year_ofObservation<=2002:
        decade3='3'   
    elif x.Year_ofObservation<=2008:
        decade3='4'
    elif x.Year_ofObservation<=2012:
        decade3='5'
    else:
        decade3='6'
    return decade3
def UnifiedImprovement(x): #to unify the improvement Types
    if x.ImprovementType<7: #Projects on Existing Road
        unifiedtype=1
    elif x.ImprovementType<12: #New Construction Projects
        unifiedtype=2
    else:
        unifiedtype=3 #Unknown Type
    return unifiedtype

In [18]:
flatdata_adjusted=flatdata_adjusted.assign(Decade_ForecastProduced=flatdata_adjusted.apply(decade_produced,axis=1))
flatdata_adjusted=flatdata_adjusted.assign(Area=flatdata_adjusted.apply(areatype,axis=1))
flatdata_adjusted=flatdata_adjusted.assign(Decade_Opening=flatdata_adjusted.apply(decade_opening,axis=1))
flatdata_adjusted=flatdata_adjusted.assign(UnifiedImprovement=flatdata_adjusted.apply(UnifiedImprovement,axis=1))
flatdata_adjusted=flatdata_adjusted.assign(Agency=flatdata_adjusted.apply(agency,axis=1))
flatdata_adjusted

,BriefDescription,InternalProjectID,ImprovementType,ImprovementType_Availability,AreaTypeFuncClass,AreaTypeFuncClass_Availability,State,Agency,TollType,TrafficForecast,ForecastYear,ForecastYear_Availability,ForecastYearType,YearForecastProduced,YearForecastProduced_Availability,ForecasterType,ForecastingType_Availability,ForecastMethodology,ForecastMethodology_Availability,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,CounterID,FunctionalClass,FunctionalClass_Availability,DiffYear_Availability,AdjustedForecast,UnemploymentRate_ForecastYear,UnemploymentRate_ForecastYear_Availability,UnemploymentRate_YearProduced,UnemploymentRate_YearProduced_Availability,UnemploymentRate_OpeningYear,UnemploymentRate_OpeningYear-Availability,DiffYear,ForecastHorizon,Error,PercentError,AbsPE,Decade_ForecastProduced,Area,Decade_Opening,UnifiedImprovement
0,NaN,2320,12,0,3,1,WI,F,No Tolls on 1+ lane,34000,2007,1,1,2006,1,1,1,1,1,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,58-22417-401399,3,1,1,34000.0,4.90,1,4.70,1,4.90,1,1,1,-2500.0,-7.352941,7.352941,5,2,4,3
1,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,30000,2011,1,1,2007,1,1,1,1,1,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,58-22419-404512,3,1,1,30000.0,7.80,1,4.90,1,7.80,1,4,4,-7800.0,-26.000000,26.000000,5,2,5,3
2,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,19000,2011,1,1,2007,1,1,1,1,1,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,58-22421-404513,3,1,1,19000.0,7.80,1,4.90,1,7.80,1,4,4,1100.0,5.789474,5.789474,5,2,5,3
3,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,21900,2011,1,1,2007,1,1,1,1,1,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,58-22423-404524,4,1,1,21900.0,7.80,1,4.90,1,7.80,1,4,4,-3500.0,-15.981735,15.981735,5,2,5,3
4,NaN,2322,12,0,3,1,WI,F,No Tolls on 1+ lane,22700,2009,1,1,2006,1,1,1,1,1,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,58-22425-401843,3,1,1,22700.0,8.60,1,4.70,1,8.60,1,3,3,-100.0,-0.440529,0.440529,5,2,5,3
5,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,2800,2011,1,1,2007,1,1,1,1,1,NaN,58-6307-2467-,NaN,58-22427-404541,2011,2500,404541.0,58-22427-404541,5,1,1,2800.0,7.80,1,4.90,1,7.80,1,4,4,-300.0,-10.714286,10.714286,5,2,5,3
6,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,33500,2011,1,1,2007,1,1,1,1,1,NaN,58-6308-2467-,NaN,58-22429-404545,2011,33500,404545.0,58-22429-404545,3,1,1,33500.0,7.80,1,4.90,1,7.80,1,4,4,0.0,0.000000,0.000000,5,2,5,3
7,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,1500,2011,1,1,2007,1,1,1,1,1,NaN,58-6309-2467-,NaN,58-22431-404547,2011,2200,404547.0,58-22431-404547,7,1,1,1500.0,7.80,1,4.90,1,7.80,1,4,4,700.0,46.666667,46.666667,5,2,5,3
8,NaN,2467,12,0,3,1,WI,F,No Tolls on 1+ lane,23800,2011,1,1,2007,1,1,1,1,1,NaN,58-6310-2467-,NaN,58-22433-404548,2011,25400,404548.0,58-22433-404548,3,1,1,23800.0,7.80,1,4.90,1,7.80,1,4,4,1600.0,6.722689,6.722689,5,2,5,3
9,NaN,2373,12,0,1,1,WI,F,No Tolls on 1+ lane,8600,2008,1,1,2006,1,1,1,1,1,NaN,58-6311-2373-,NaN,58-22435-510108,2008,6600,510108.0,58-22435-510108,3,1,1,8600.0,4.90,1,4.70,1,4.90,1,2,2,-2000.0,-23.255814,23.255814,5,1,4,3


## Step 7. Aggregating on Internal Project ID for Project Level Analysis
The flatdata file contains the segment level data, which can be accumulated over the project level by the Unique Identifier for each projects. The Identified is the Internal Project ID.

In [23]:
#Taking the average of the Forecasted Volume and Actual Volume over all the segments.
averagecount=flatdata_adjusted['Count'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='AverageCount')
averageforecast=flatdata_adjusted['AdjustedForecast'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='AverageForecast')

#Project Improvement Type and Functional Class taken to be the Improvement Type with the Highest count among the segments.
improvement_type=flatdata_adjusted['ImprovementType'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='ImprovementType')
func_class=flatdata_adjusted['FunctionalClass'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='Func_Class')

#Forecast methodology is the same across the segments in a project and are coded as integer values. Hence taking the mean.
forecast_method=flatdata_adjusted['ForecastMethodology'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='Forecast_Method')

#Variables related to years- year forecast produced, year project is forecasted to open, year of observation.
#These values are taken to be the highest occuring onces.
year_produced=flatdata_adjusted['YearForecastProduced'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='YearForecastProduced')
year_open=flatdata_adjusted['ForecastYear'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='ForecastYear')
year_count=flatdata_adjusted['Year_ofObservation'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='Year_ofObservation')
diff_year=flatdata_adjusted['DiffYear'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='Diff_Year')

#Unemployment Rates are taken as average across the segments. 
#Since the rates are taken on a state level, taking the mean won't result in exceptional values.
unemp_forecast=flatdata_adjusted['UnemploymentRate_ForecastYear'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='UnemploymentRate_ForecastYear')
unemp_produced=flatdata_adjusted['UnemploymentRate_YearProduced'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='UnemploymentRate_YearProduced')
unemp_opening=flatdata_adjusted['UnemploymentRate_OpeningYear'].groupby(flatdata_adjusted['InternalProjectID']).mean().to_frame(name='UnemploymentRate_OpeningYear')

#Area Type taken to be the minimum value
area_type=flatdata_adjusted['Area'].groupby(flatdata_adjusted['InternalProjectID']).min().to_frame(name='Area_Type')

#Forecaster Type (State DOT, MPO, Consultant etc.) is coded in integers. So, taking the mean across the segments.
agency=flatdata_adjusted['ForecasterType'].groupby(flatdata_adjusted['InternalProjectID']).agg(lambda x:x.value_counts().index[0]).to_frame(name='ForecasterType')

In [21]:
#Merged the previously created dataframes and added new project information that went missing in the excel file manually.
#Project_Flatdata.csv contains all the projects that passed through the filters and conditions
#The logic behind getting rid of the outliers and sifting through the segments is explained in the Interim Report Chapter 3.
project_error=pd.read_csv('Project_Flatdata.csv')

In [22]:
project_error=project_error.assign(Decade_ForecastProduced=project_error.apply(decade_produced,axis=1))
project_error=project_error.assign(Decade_Opening=project_error.apply(decade_opening,axis=1))
project_error=project_error.assign(UnifiedImprovementType=project_error.apply(UnifiedImprovement,axis=1))
project_error=project_error.assign(DiffYear=project_error['ForecastYear']-project_error['YearForecastProduced'])
project_error['ForecastHorizon'] = np.where(project_error['DiffYear']<=5, project_error['DiffYear'], '5+')